# BL-Cam : Time Series analysis avec AICO data

### June Parsons | 20191216 | 20200507+08 | 20200521+22 | 20200525+26

---

![XX-Cyg ](tangerine_smooth_slick_yum_lc_xxcyg.png)

---

This is a sample lightcurve obtained using DRACO from historical data of XX-Cyg obtained by the Allan I. Carswell Observatory in September of 2017 during the writers first night at the observatory which was their first time looking through a telescope or partaking in research. It is markedly a monumental moment for the writer and became the inception of the writers valued relationship with the observatory and research into variable stars and broad dedication to observational and theoretical astrophysics!

# Dependencies 
---

DRACO currently runs off of the following dependencies: 

#### Numpy ()
#### Matplotlib ()
#### Photutils ()
#### Astropy ()
#### Astroquery () *
#### ccdproc ()
#### Scipy () *
#### Scikit/Skimage () *
#### Sklearn () *

We'd like to extend immense gratitude to the teams behind these wonderful opensource projects; without their contribution DRACO would not be able to function. We would also like to thank the Transiting exoplanet survey satellite (TESS) team for their dedication and thorough open access data.

* this list may not be up to date and may include a few more or less dependencies.


# Non-code related next steps:
---
Next steps include running script through all of TESS XXCyg data and saving times of max light. Comparing the time of max light to data obtained at AICO. Running Fourier analysis through ~18000 historic data points, obtaining times of max light and comparing to the properties of all other data sources.

Planning for future TESS data.

# Input for this pipeline
___
This pipeline takes one input to run for TESS observations and is discussed in section 1 of the code. 
The future of this pipeline will include small modifications that receive input data from the Allan I. Carswell observatory (AICO); the nature of the input has yet to be determined. The input may be timeseries data in a table or an unprocessed data set.

# 0.1 Import code libraries used within the data pipeline:
---


In [1]:
# %matplotlib notebook
#interactive but cut off
%matplotlib ipympl
# %matplotlib widget
#window
# %matplotlib qt 
# static and inline
# %matplotlib inline
# Import to clean up package warning information about future depreciation in astropy
# Import to keep track of computing time
import datetime
import warnings
warnings.filterwarnings('ignore')
#########
from astropy.io import fits
from astropy.table import Table
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import lightkurve as lk
from lightkurve.lightcurve import TessLightCurve as tlc
from photutils import aperture_photometry, RectangularAperture, RectangularAnnulus
import dracoOP2 as dr2


# 0.2 Initialize time keeping for performance testing : 
___

In [2]:
# Initialize time tracker variables - and print start time
START_DATE_TIME = datetime.datetime.now()

print('\nStarting time: ', START_DATE_TIME)


Starting time:  2020-05-26 14:30:52.946105


# 1 Read in the Target Pixel File (TPF) :
___
A Target Pixel File, obtained from TESS via the STScl | MAST (https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html), is a FITS file that contains a 'stamp' or pixel cutout surrounding and object for the duration of a sector. Objects observed in multiple sectors will have a TPF for each sector observed (https://archive.stsci.edu/files/live/sites/mast/files/home/missions-and-data/active-missions/tess/_documents/EXP-TESS-ARC-ICD-TM-0014.pdf).

TPF's are created from Full Frame Images (FFI's) and have cosmic ray corrections applied (stored in HDU[3] as seen in the output below), the primary HDU (HDU[0]) of the TPF FITS file contains primary information about the target as well as the star's Target ID (TID). The second HDU (HDU[1]) contains the stamp series of the TPF, for BL Cam sector 16 the series is 17765 11x11 pixel stamps each with an accompanied timestamp of the TESS calibrated Barycentric Julian Date (tBJD) for each stamp's observation time. The third HDU (HDU[2]) contains the aperture mask used by the TESS pipeline, which is automatically generated to encompass the target and used by the pipeline to produce the sample lightcurves generated for the pipeline to do rudimentary searches for exoplanet transits. 

In [3]:
# Open the TPF and print important info 
data_file = "AICO_XX_Cyg_timestrings.csv" #xx-cyg

tsf = Table.read(data_file)
aico_hjds = tsf['ï»¿HJD'].data
mag = tsf['Mag'].data

# 4 Constructing a DRACO format lightcurve table with Astropy
___
The **lightKurve** *object* format is not ideal for further analysis outside of basic exoplanet transit analysis. We then grab our photometric sums and our time values and construct an **astropy** table containing the resulting timeseries and a header which is currently empty.This data can be exported as a fits file, but the acting code is commented out.

In [4]:
# Construct Astropy Table column objects that contain the flux values and their associated times
c1 = fits.Column(name='Mag', array = mag, format='D')
c2 = fits.Column(name='TIME', array = aico_hjds, format='D')

# Combine the columns into a fits HDU (Header Data Unit)
table_hdu = fits.BinTableHDU.from_columns([c1, c2])

# Produce the fits Header
hdr = fits.Header()
# Set the fits Header as the Primary HDU
h_primary = fits.PrimaryHDU(header=hdr)
# Combine into a single fits object
hdul = fits.HDUList([h_primary, table_hdu])

# Write the fits object to a file in the root of the python pipeline
# In this case, where this notebook is run
# hdul.writeto('BLCam_june_lc.fits', overwrite = True)

# 5 Describing Fourier analysis functions - DRACO
___
For the purposes of explanation many pieces of **DRACO** have been dissected and inserted into this script. A very important piece for analysis and modeling of the lightcurve is **Fournax** *(Fournax is an abbreviation of Fourier numerical astronomy extension, its name is a backronym styled to match the constellation 'fornax'.)* . Fournax performs very simple operations on the timeseries data to create a smooth yet accurate data fit model. The first step is to perform a real valued fast Fourier transform on the flux values of the time series to obtain a Fourier series represented by the array 'f'. The function takes a parameter 'terms' which represents the number of terms of the Fourier series to retain, fournax then zeros out the higher terms in the series and then performs and inverse real valued fast Fourier transform to obtain a smoothed function fit to the time series data.

*pour* is a function that converts a time series into a *power spectral density* (PSD) series stored in the array 'p'.
This *periodogram* estimation of the spectral density of the timeseries function  *pour* uses is due to the data being sampled at 2 minute intervals as opposed to a continuous function of data. The *periodogram* PSD is given by:
$$ P(x(t)) = |fft(x(t))|^2 $$ 
where $fft(x(t))$ is the fast fourier transform of the flux at a given time where $x(t)$ is the timeseries. The PSD is contained in frequency space in units of $days^{−1}$  .

In [5]:
# Homebuilt Fourier analysis - Fourier fitting and local max recording
# Compute ifft()
from scipy.signal import find_peaks, periodogram, spectrogram, lombscargle, welch
from astropy.timeseries import TimeSeries, LombScargle

def fournax(x, y, terms):
    # Fournax is an abbreviation of Fourier numerical astronomy extension, its name is a backronym styled to match the constellation 'fornax'.
    # Compute real valued Fourier transform
    f = np.fft.rfft(y)
    # Null or zero coefficients above ammount of series "terms"
    # This corresponds to undesired high-frequency terms
    f[terms+1:] = 0
    # Collapse back into function space, result is smoothed Fourier curve
    F = np.fft.irfft(f)
    return F



def pour(y):

    f = np.fft.fft(y)
    p = np.square(np.abs(f))    
    return p


# 6 Preparing time series for Fourier fitting and power spectral density analysis
___
We assign the *x*-axis to hold our time values and the *y*-axis to hold our flux amplitude values. The terms *1-3* parameters are to be input into *fournax* and define how many terms of the Fourier transform series *fournax* should retain. The values we choose are done based on inspection as to the number of terms the removes the scatter from the data points to a smoothed curve but is not too reduced so that small scale variations in the curve are removed. *terms3* is our candidate model, while *terms* and *terms2* are comparison curves to analyze how removing more terms affects the accuracy of the curve.


In [7]:
x = aico_hjds #table_hdu['TIME']#[0:round(len(lc_june.time)/4)]#-300]#[0:4000]
y = mag #table_hdu['MAG']#[0:round(len(lc_june.time)/4)]#-300]#[0:4000]

# Frequency terms to retain, this is currently hand coded and different for every star so far
# Probably depends on main frequency and data length.
terms = 3000
Fcurve = fournax(x, y, terms)

terms2 = 4000
Fcurve2 = fournax(x, y, terms2)

terms3 = 5000 # bl cam
Fcurve3 = fournax(x, y, terms3)


# 7.1 Assembling the power spectral density data 
___
We now take our raw *y* axis data *(as opposed to Fcurve3 which stores our modeled y-axis data)* and imput it into the **DRACO** *pour* function to obtain the *y* axis values for our PSD data. Next we assemble our frequency domain in units of days. The time step comes from each data point being obtained from the 2-minute cadence stamps in the *target pixel file* from TESS. We then call the **numpy** *fftfreq* constructor which takes the dataset length and the timestep and returns a frequency array for the *x* axis.
## 7.2 Finding the peaks of the timeseries*(times of max light)* and PSD
___
The mean of both data sets is computed, we will search for peaks above a scale of this value in order to avoid noise on the floor of both series. Next we utilize the find_peaks function from **Scipy** *.signal*
Local maxima of the PSD are obtained using the 'scipy.signal.find_peaks' function, where only local maxima above the scaled mean are considered.

In [8]:
# Set up a Fourier power spectra from photometric amplitude values (TESS flux values)
Pxx = pour(Fcurve3) 

# Build an array of frequencies to plot against
freqs = np.arange(1, len(Pxx), 1)

# Using the np.fft frequency array constructor, admit that June sucks and 
# June's bulky solutions suck compared to the glorious numpy
# timestep in days
timestep = 2 / 1440
n = len(Fcurve3) 
freq = np.fft.fftfreq(n, d=timestep)

# Compute mean to find peaks above
h = np.mean(Fcurve3)
hp = np.mean(Pxx)
peaks, _ = find_peaks(Fcurve3, height =  1.1 * h)
peaks_p, _ = find_peaks(Pxx, height = hp ) 


# 8 Plotting
___
The raw data (light blue) and Fourier smoothed fit (olive) of the XX-Cyg lightcurve from TESS sector 16 at 2 minute cadence from our target pixel file (TPF) photometry. Times of max light are denoted by blue open circular markers and as stated above are located using the 'scipy.signal.find_peaks' function.

The PSD plot has the frequency peaks highlighted using filled blue circle markers.

The times of max light are printed below along with a sample of the peak frequencies identified. Due to the noisy nature of frequency diagrams, multiple peak frequencies may be identified for a given frequency peaks. Additional logic is thus required to be written into DRACO to accommodate for this phenomenon.

In [13]:
# Visualization (Light curve)
fig, ax = plt.subplots()
# Let's define a title for the figure.
fig.suptitle("XX-Cyg Lightcurve - AICO, lifetime data")
ax.plot(x, y, color='lightblue', label = 'raw data', linewidth=0.5)
ax.plot(x, Fcurve3, '.',  color='olive',label = ("'rFFT' series fit (%d terms)" % terms3), linewidth=0.01) # bl cam
# ax.plot(x[peaks], Fcurve3[peaks], 'o', label = ("'rFFT' series peaks" ), markersize=5, fillstyle='none')
ax.grid(True, color='dimgray', linestyle='--', linewidth=0.1)
ax.set_axisbelow(True)
ax.set_ylabel('calibrated TESS flux')
ax.set_xlabel('time -BJDs')
plt.gca().invert_yaxis()
# ax.legend()

# Visualization (Power series)
fig, ax = plt.subplots()
# Let's define a title for the figure.
fig.suptitle("XX-Cyg Power series - AICO, lifetime data")
ax.semilogy(freq[peaks_p], Pxx[peaks_p], '.', label = ("'power' series peaks" )) #[0:600]
ax.semilogy(freq, Pxx, label = ("'power' series fit (%d terms)" % terms3), color='rebeccapurple', linestyle='-', linewidth=0.5)
ax.grid(True, color='dimgray', linestyle='--', linewidth=0.1)
ax.set_axisbelow(True)
ax.set_ylabel('power')
ax.set_xlabel('frequency ($days^{-1}$)')
# ax.legend()

# Output times of max light and main frequencies for quick analysis
print('Times of max light: ', x[peaks])
print('Main frequencies: ', freq[peaks_p])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Times of max light:  [2455427.815 2455453.608 2455453.752 2455778.625 2455785.662 2455785.668
 2455785.674 2455785.678 2455785.684 2455785.689 2455785.695 2455796.755
 2455799.729 2455839.742 2455839.747 2455841.719 2455859.68  2456154.609
 2456154.741 2456154.747 2456154.751 2456154.756 2456154.76  2456154.762
 2456154.764 2456154.767 2456154.769 2456154.771 2456154.773 2456154.778
 2456154.78  2456154.785 2456154.789 2456154.793 2456154.796 2456154.799
 2456155.588 2456155.591 2456155.593 2456155.719 2456155.726 2456155.728
 2456155.73  2456155.732 2456163.682 2456163.733 2456163.735 2456163.737
 2456163.74  2456163.807 2456163.809 2456163.821 2456163.823 2456163.826
 2456163.828 2456163.83  2456163.832 2456163.834 2456163.839 2456163.844
 2456163.847 2456168.797 2456168.809 2456168.813 2456168.818 2456168.828
 2456168.83  2456168.832 2456182.554 2456182.783 2456182.785 2456183.621
 2456183.623 2456183.626 2456183.628 2456183.632 2456183.636 2456183.64
 2456183.644 2456183.646 245618

# 9 Preparing times for export
___
Using the **Astropy** *.time* library we create a time list object in memory. The format is set to julian date with UTC time. The HJD time offset of $2457000$ is applied and the table is printed out with fits time styling to verify success.

In [ ]:
from astropy.time import Time
times_of_max_light = Time(x[peaks], format='jd', scale='utc')
print(times_of_max_light.fits)

# 10 Assembling the output table
___
A table object is created named *toml* (times of max light). *toml* stores the raw TESS times of max light, the HJD formatted times of max light, and the fits time formatted times of max light. 
The times are put into a new lightcurve table *lc_blcam* and the corresponding flux values for the times of max light are included. The time table is printed to the notebook for further verification and the table can be written to the directory with the uncommenting of a line of code.

In [ ]:
toml = Table()
toml["tBJD"] = x[peaks]
toml['HJD'] = times_of_max_light
toml['fits'] = times_of_max_light.fits

# toml.write('blcam_toml_1.fits', overwrite=True)
# toml.write('blcam_toml_1.csv', overwrite=True)
lc_blcam = toml
lc_blcam['dFlux'] = y[peaks]
# lc_blcam.write('blcam_lc_s15.csv', overwrite=True)

toml.show_in_notebook()

# 11 The folded lightcurve
___
While this process is still a work in progress, we are interested in making a detailed model of the BL-Cam cycle. To accomplish this we take our complete list of times and flux and input them into the **Astropy** Timeseries object constructor. We've done this before in many different ways that made sense to the steps at hand; but future iterations of this pipeline will be streamlined to jump straight to this format of storing lightcurve timeseries data and DRACO functions rewritten to accommodate this new format. 

We set the epoch to fold our series by to be the first time of max light identified. We then take advantage of the timeseries fold function that is built into the **Astropy** timeseries object class to obtain our folded timeseries.

Our last step is to plot the result using a scatterplot of the points in our folded timeseries object.

In [ ]:
times = Time(x , format='jd', scale='utc')
print(len(np.array(y)))
ts = TimeSeries(data= [y], time=times)
period = 0.0390976 * u.d
# period = (1/25.577) * u.d
epoch = times_of_max_light[0]
ts_f = ts.fold(period=period, midpoint_epoch = epoch) # , epoch_time = epoch)  
print(ts_f.colnames)

# Visualization
fig, ax = plt.subplots()

fig.suptitle("BL-Cam Folded Series - Sector 15, 2 Min Cadence")
ax.scatter(ts_f['time'].jd, ts_f['col0'], label = ('Folded LightCurve'), s = 0.1)

ax.grid(True, color='dimgray', linestyle='--', linewidth=0.1)
ax.set_axisbelow(True)
ax.set_ylabel('Flux')
ax.set_xlabel('Time')
ax.legend()


## 12 Computing the run time of the pipeline
___


In [ ]:
# Print the end time and total computing time
END_DATE_TIME = datetime.datetime.now()
print('\nEnding time: ', END_DATE_TIME)
print("Time elapsed: ", (END_DATE_TIME - START_DATE_TIME))

## References
[1]: Thomas E. Obert, Joseph E. Rodriguez, Knicole D Colon, et al. 	arXiv:1608.00618 [astro-ph.EP] https://arxiv.org/abs/1608.00618

[2]: Stéfan van der Walt, S. Chris Colbert and Gaël Varoquaux. The NumPy Array: A Structure for Efficient Numerical Computation, Computing in Science & Engineering, 13, 22-30 (2011), DOI:10.1109/MCSE.2011.37 

[3]: Travis E. Oliphant. A guide to NumPy, USA: Trelgol Publishing, (2006).

[4]: [1]P. Virtanen, R. Gommers, T. E. Oliphant, M. Haberland, T. Reddy, D. Cournapeau, E. Burovski, P. Peterson, W. Weckesser, J. Bright, S. J. van der Walt, M. Brett, J. Wilson, K. Jarrod Millman, N. Mayorov, A. R. J. Nelson, E. Jones, R. Kern, E. Larson, C. Carey, İ. Polat, Y. Feng, E. W. Moore, J. Vand erPlas, D. Laxalde, J. Perktold, R. Cimrman, I. Henriksen, E. A. Quintero, C. R. Harris, A. M. Archibald, A. H. Ribeiro, F. Pedregosa, P. van Mulbregt, and S. 1. 0 Contributors, Nature Methods 17, 261 (2020).

[5]: Fernando Pérez and Brian E. Granger. IPython: A System for Interactive Scientific Computing, Computing in Science & Engineering, 9, 21-29 (2007), DOI:10.1109/MCSE.2007.53

[6]: John D. Hunter. Matplotlib: A 2D Graphics Environment, Computing in Science & Engineering, 9, 90-95 (2007), DOI:10.1109/MCSE.2007.55

[7]: Stéfan van der Walt, Johannes L. Schönberger, Juan Nunez-Iglesias, François Boulogne, Joshua D. Warner, Neil Yager, Emmanuelle Gouillart, Tony Yu and the scikit-image contributors. scikit-image: Image processing in Python, PeerJ 2:e453 (2014)

[8]: Fabian Pedregosa, Gaël Varoquaux, Alexandre Gramfort, Vincent Michel, Bertrand Thirion, Olivier Grisel, Mathieu Blondel, Peter Prettenhofer, Ron Weiss, Vincent Dubourg, Jake Vanderplas, Alexandre Passos, David Cournapeau, Matthieu Brucher, Matthieu Perrot, Édouard Duchesnay. Scikit-learn: Machine Learning in Python, Journal of Machine Learning Research, 12, 2825-2830 (2011)

[9]: Astropy Collaboration, T. P. Robitaille, E. J. Tollerud, P. Greenfield, M. Droettboom, E. Bray, T. Aldcroft, M. Davis, A. Ginsburg, A. M. Price-Whelan, W. E. Kerzendorf, A. Conley, N. Crighton, K. Barbary, D. Muna, H. Ferguson, F. Grollier, M. M. Parikh, P. H. Nair, H. M. Unther, C. Deil, J. Woillez, S. Conseil, R. Kramer, J. E. H. Turner, L. Singer, R. Fox, B. A. Weaver, V. Zabalza, Z. I. Edwards, K. Azalee Bostroem, D. J. Burke, A. R. Casey, S. M. Crawford, N. Dencheva, J. Ely, T. Jenness, K. Labrie, P. L. Lim, F. Pierfederici, A. Pontzen, A. Ptak, B. Refsdal, M. Servillat, and O. Streicher, 558, A33 (2013).

[10]: A. M. Price-Whelan, B. M. Sipőcz, H. M. Günther, P. L. Lim, S. M. Crawford, S. Conseil, D. L. Shupe, M. W. Craig, N. Dencheva, A. Ginsburg, J. T. VanderPlas, L. D. Bradley, D. Pérez-Suárez, M. de Val-Borro, (Primary Paper Contributors, T. L. Aldcroft, K. L. Cruz, T. P. Robitaille, E. J. Tollerud, (Astropy Coordination Committee, C. Ardelean, T. Babej, Y. P. Bach, M. Bachetti, A. V. Bakanov, S. P. Bamford, G. Barentsen, P. Barmby, A. Baumbach, K. L. Berry, F. Biscani, M. Boquien, K. A. Bostroem, L. G. Bouma, G. B. Brammer, E. M. Bray, H. Breytenbach, H. Buddelmeijer, D. J. Burke, G. Calderone, J. L. Cano Rodríguez, M. Cara, J. V. M. Cardoso, S. Cheedella, Y. Copin, L. Corrales, D. Crichton, D. DAvella, C. Deil, É. Depagne, J. P. Dietrich, A. Donath, M. Droettboom, N. Earl, T. Erben, S. Fabbro, L. A. Ferreira, T. Finethy, R. T. Fox, L. H. Garrison, S. L. J. Gibbons, D. A. Goldstein, R. Gommers, J. P. Greco, P. Greenfield, A. M. Groener, F. Grollier, A. Hagen, P. Hirst, D. Homeier, A. J. Horton, G. Hosseinzadeh, L. Hu, J. S. Hunkeler, Ž. Ivezić, A. Jain, T. Jenness, G. Kanarek, S. Kendrew, N. S. Kern, W. E. Kerzendorf, A. Khvalko, J. King, D. Kirkby, A. M. Kulkarni, A. Kumar, A. Lee, D. Lenz, S. P. Littlefair, Z. Ma, D. M. Macleod, M. Mastropietro, C. McCully, S. Montagnac, B. M. Morris, M. Mueller, S. J. Mumford, D. Muna, N. A. Murphy, S. Nelson, G. H. Nguyen, J. P. Ninan, M. Nöthe, S. Ogaz, S. Oh, J. K. Parejko, N. Parley, S. Pascual, R. Patil, A. A. Patil, A. L. Plunkett, J. X. Prochaska, T. Rastogi, V. Reddy Janga, J. Sabater, P. Sakurikar, M. Seifert, L. E. Sherbert, H. Sherwood-Taylor, A. Y. Shih, J. Sick, M. T. Silbiger, S. Singanamalla, L. P. Singer, P. H. Sladen, K. A. Sooley, S. Sornarajah, O. Streicher, P. Teuben, S. W. Thomas, G. R. Tremblay, J. E. H. Turner, V. Terrón, M. H. van Kerkwijk, A. de la Vega, L. L. Watkins, B. A. Weaver, J. B. Whitmore, J. Woillez, V. Zabalza, and (Astropy Contributors, 156, 123 (2018).
